## Imports and Load Data

In [29]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, Flatten, Dense, Conv1D, MaxPooling1D
from tensorflow.keras.models import save_model, load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
tqdm.pandas()
import pickle
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


This code assumes that the data is preprocessed including clustering, data augmentation and dimension reduction (if required)

In [3]:
with open('/content/drive/MyDrive/ml_project/spectral_X_train_flat.pkl','rb') as f:
    X_train_flat = pickle.load(f)

with open('/content/drive/MyDrive/ml_project/spectral_X_test_flat.pkl','rb') as f:
    X_test_flat = pickle.load(f)

with open('/content/drive/MyDrive/ml_project/spectral_X_train_cood.pkl','rb') as f:
    X_train_cood = pickle.load(f)

with open('/content/drive/MyDrive/ml_project/spectral_X_test_cood.pkl','rb') as f:
    X_test_cood = pickle.load(f)

with open('/content/drive/MyDrive/ml_project/spectral_y_train.pkl','rb') as f:
    y_train = pickle.load(f)

with open('/content/drive/MyDrive/ml_project/spectral_y_test.pkl','rb') as f:
    y_test = pickle.load(f)

In [4]:
X_train_flat.shape

(20971, 15, 150)

In [5]:
# 1D CNN and BiLSTM
X_train = X_train_flat
X_test = X_test_flat

# 2D CNN
# X_train = X_train_cood
# X_test = X_test_cood

In [6]:
#X_train, X_test, y_train, y_test = train_test_split(data_pca, labels, test_size=0.2, random_state=42)

label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y_train)
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


In [7]:
len(label_encoder.classes_)

1624

In [8]:
X = np.concatenate((X_train, X_test), axis=0)
Y = np.concatenate((y_train_encoded, y_test_encoded), axis=0)

In [9]:
y_series = pd.Series(Y)

# Calculate the class frequencies
class_frequencies = y_series.value_counts()

# Identify classes with frequency more than 15
selected_classes = class_frequencies[class_frequencies > 19].index

# Filter X and Y based on selected classes
filtered_x = X[y_series.isin(selected_classes)]
filtered_y = Y[y_series.isin(selected_classes)]

# If you want to convert Y back to a NumPy array
filtered_y = np.array(filtered_y)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(filtered_x, filtered_y, test_size=0.2, random_state=42)

In [11]:
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y_train)
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [12]:
len(label_encoder.classes_)

212

## Classification Models

### 1D CNN

In [35]:
# Building the 1D CNN Model
model = Sequential([
    Conv1D(64, kernel_size=4, activation='relu', input_shape=(15, 150)),
    MaxPooling1D(pool_size=3),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])
checkpoint_filepath = '/content/1dcheckpoint.weights.h5'
topk = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10)
mcp_save = ModelCheckpoint(checkpoint_filepath, save_best_only=True, save_weights_only=True, monitor='val_'+topk.name, mode='max')

# Compile and Train
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics= [topk]
              )

model.fit(X_train, y_train_encoded, epochs=150, batch_size=32, validation_data=(X_test, y_test_encoded), callbacks=[mcp_save])

# Load best model & Evaluate
model.load_weights(checkpoint_filepath)
test_loss, test_acc = model.evaluate(X_test, y_test_encoded)
print(f'Test accuracy: {test_acc}')

Epoch 1/150
123/123 [==============================] - 1s 5ms/step - loss: 5.3057 - sparse_top_k_categorical_accuracy: 0.0756 - val_loss: 5.1434 - val_sparse_top_k_categorical_accuracy: 0.0824
Epoch 2/150
123/123 [==============================] - 0s 4ms/step - loss: 4.9769 - sparse_top_k_categorical_accuracy: 0.1135 - val_loss: 4.9984 - val_sparse_top_k_categorical_accuracy: 0.0855
Epoch 3/150
123/123 [==============================] - 1s 4ms/step - loss: 4.8450 - sparse_top_k_categorical_accuracy: 0.1534 - val_loss: 4.9189 - val_sparse_top_k_categorical_accuracy: 0.1302
Epoch 4/150
123/123 [==============================] - 0s 4ms/step - loss: 4.7194 - sparse_top_k_categorical_accuracy: 0.1992 - val_loss: 4.8875 - val_sparse_top_k_categorical_accuracy: 0.1516
Epoch 5/150
123/123 [==============================] - 0s 4ms/step - loss: 4.5580 - sparse_top_k_categorical_accuracy: 0.2554 - val_loss: 4.8354 - val_sparse_top_k_categorical_accuracy: 0.1872
Epoch 6/150
123/123 [==============

In [36]:
model.save('mediapipe_spectral_1dcnn')
reconstructed_model = load_model("mediapipe_spectral_1dcnn")
test_loss, test_acc = reconstructed_model.evaluate(X_test, y_test_encoded)
print(f'Test accuracy: {test_acc}')

31/31 [==============================] - 0s 2ms/step - loss: 15.9361 - sparse_top_k_categorical_accuracy: 0.5239
Test accuracy: 0.523906409740448


In [39]:
!cp /content/1dcheckpoint.weights.h5 /content/drive/MyDrive/ml_project/1dcheckpoint.weights.h5
!cp -r /content/mediapipe_spectral_1dcnn /content/drive/MyDrive/ml_project/mediapipe_spectral_1dcnn

### Stacked BiLSTM

In [41]:
# Building the Stacked BiLSTM Model
model = Sequential([
    Bidirectional(LSTM(128, return_sequences=True), input_shape=(15, 150)),
    Dropout(0.1),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.1),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(32, return_sequences=True)),
    Bidirectional(LSTM(16, return_sequences=True)),
    Flatten(),
    Dense(1000, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])


checkpoint_filepath = '/content/bilstmcheckpoint.weights.h5'
topk = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10)
mcp_save = ModelCheckpoint(checkpoint_filepath, save_best_only=True, save_weights_only=True, monitor='val_'+topk.name, mode='max')

# Compile and Train
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics= [topk]
              )

model.fit(X_train, y_train_encoded, epochs=100, batch_size=32, validation_data=(X_test, y_test_encoded), callbacks=[mcp_save])

# Load best model & Evaluate
model.load_weights(checkpoint_filepath)
test_loss, test_acc = model.evaluate(X_test, y_test_encoded)
print(f'Test accuracy: {test_acc}')

Epoch 1/100
123/123 [==============================] - 20s 53ms/step - loss: 5.1668 - sparse_top_k_categorical_accuracy: 0.0781 - val_loss: 5.0281 - val_sparse_top_k_categorical_accuracy: 0.0783
Epoch 2/100
123/123 [==============================] - 2s 18ms/step - loss: 4.9585 - sparse_top_k_categorical_accuracy: 0.0865 - val_loss: 4.9989 - val_sparse_top_k_categorical_accuracy: 0.0722
Epoch 3/100
123/123 [==============================] - 2s 19ms/step - loss: 4.9095 - sparse_top_k_categorical_accuracy: 0.0974 - val_loss: 4.9597 - val_sparse_top_k_categorical_accuracy: 0.0824
Epoch 4/100
123/123 [==============================] - 2s 19ms/step - loss: 4.8615 - sparse_top_k_categorical_accuracy: 0.1112 - val_loss: 4.9464 - val_sparse_top_k_categorical_accuracy: 0.0916
Epoch 5/100
123/123 [==============================] - 2s 19ms/step - loss: 4.8345 - sparse_top_k_categorical_accuracy: 0.1175 - val_loss: 4.9263 - val_sparse_top_k_categorical_accuracy: 0.1190
Epoch 6/100
123/123 [========

In [42]:
model.save('mediapipe_spectral_bilstm')
reconstructed_model = load_model("mediapipe_spectral_bilstm")
test_loss, test_acc = reconstructed_model.evaluate(X_test, y_test_encoded)
print(f'Test accuracy: {test_acc}')

31/31 [==============================] - 3s 8ms/step - loss: 5.8314 - sparse_top_k_categorical_accuracy: 0.6836
Test accuracy: 0.6836215853691101


In [43]:
!cp /content/bilstmcheckpoint.weights.h5 /content/drive/MyDrive/ml_project/bilstmcheckpoint.weights.h5
!cp -r /content/mediapipe_spectral_bilstm /content/drive/MyDrive/ml_project/mediapipe_spectral_bilstm